In [109]:
# --------------------------
# Estructuras base
# --------------------------
class Node:
    def __init__(self, distribution, name):
        self.name = name
        self.distribution = distribution
        self.parents = []
        self.children = []

    def set_parents(self, parents):
        self.parents = parents
        for p in parents:
            p.children.append(self)

class DiscreteDistribution:
    def __init__(self, probabilities):
        self.probabilities = probabilities
    def probability(self, value):
        return self.probabilities[value]

class ConditionalProbabilityTable:
    def __init__(self, table, parents):
        self.table = table
        self.parents = parents  # lista de nodos
        self.data = {}
        for row in table:
            *parent_vals, value, prob = row
            key = tuple(parent_vals)
            if key not in self.data:
                self.data[key] = {}
            self.data[key][value] = prob

    def probability(self, value, parent_values):
        key = tuple(parent_values)
        if key not in self.data or value not in self.data[key]:
            raise KeyError(f"Clave no encontrada en CPT: padres={key}, valor={value}")
        return self.data[key][value]


# --------------------------
# Red bayesiana
# --------------------------
from itertools import product

class BayesianNetwork:
    def __init__(self):
        self.nodes = []
        self.name_to_node = {}

    def add_state(self, *nodes):
        for node in nodes:
            self.nodes.append(node)
            self.name_to_node[node.name] = node

    def add_edges(self, *edges):
        for i in range(0, len(edges), 2):
            parent, child = edges[i], edges[i+1]
            self.name_to_node[child.name].set_parents(
                self.name_to_node[child.name].parents + [self.name_to_node[parent.name]]
            )

    def bake(self):
        pass

    def predict_proba(self, evidence):
        domains = {}

        for node in self.nodes:
            if isinstance(node.distribution, DiscreteDistribution):
                domains[node.name] = list(node.distribution.probabilities.keys())
            else:
                sample_key = next(iter(node.distribution.data))
                domains[node.name] = list(node.distribution.data[sample_key].keys())

        results = []
        total = 0
        probs = {}

        for values in product(*[domains[n.name] for n in self.nodes]):
            assignment = dict(zip([n.name for n in self.nodes], values))
            if not all(assignment.get(k) == v for k, v in evidence.items()):
                continue

            p = 1.0
            for node in self.nodes:
                val = assignment[node.name]
                if isinstance(node.distribution, DiscreteDistribution):
                    p *= node.distribution.probability(val)
                else:
                    parent_vals = [assignment[p.name] for p in node.parents]
                    p *= node.distribution.probability(val, parent_vals)
            probs[tuple(assignment[n.name] for n in self.nodes)] = p
            total += p

        for node in self.nodes:
            if node.name in evidence:
                results.append(evidence[node.name])
            else:
                dist = {}
                for k, p in probs.items():
                    val = k[[n.name for n in self.nodes].index(node.name)]
                    dist[val] = dist.get(val, 0) + p
                for val in dist:
                    dist[val] /= total
                results.append(DiscreteDistribution(dist))

        return results

    def probability(self, observation):
        p = 1.0
        for node, value in zip(self.nodes, observation):
            if isinstance(node.distribution, DiscreteDistribution):
                p *= node.distribution.probability(value)
            else:
                parent_vals = [
                    observation[self.nodes.index(p)] for p in node.parents
                ]
                p *= node.distribution.probability(value, parent_vals)
        return p


In [110]:
# Definir nodos
#from crear_red import *
lluvia = Node(DiscreteDistribution({
    "ninguna": 0.7,
    "suave": 0.2,
    "fuerte": 0.1
}), name="lluvia")

In [111]:
mantenimiento = Node(ConditionalProbabilityTable([
    ["ninguna","si",0.4],
    ["ninguna","no",0.6],
    ["suave","si",0.7],
    ["suave","no",0.3],
    ["fuerte","si",0.9],
    ["fuerte","no",0.1],
], [lluvia]), name="mantenimiento")

In [112]:
bus = Node(ConditionalProbabilityTable([
    ["ninguna","si","a tiempo", 0.8],
    ["ninguna","si","retrasada", 0.2],
    ["ninguna","no","a tiempo", 0.9],
    ["ninguna","no","retrasada", 0.1],
    ["suave","si","a tiempo", 0.6],
    ["suave","si","retrasada", 0.4],
    ["suave","no","a tiempo", 0.7],
    ["suave","no","retrasada", 0.3],
    ["fuerte","si","a tiempo", 0.3],
    ["fuerte","si","retrasada", 0.7],
    ["fuerte","no","a tiempo", 0.5],
    ["fuerte","no","retrasada", 0.5]
], [lluvia, mantenimiento]), name="bus")

In [113]:
cita = Node(ConditionalProbabilityTable([
    ["a tiempo","atendida", 0.95],
    ["a tiempo","perdida", 0.05],
    ["retrasada","atendida", 0.6],
    ["retrasada","perdida", 0.4]
], [bus]), name="cita")

In [114]:
# Crear la red
modelo = BayesianNetwork()
modelo.add_state(lluvia, mantenimiento, bus, cita)
modelo.add_edges(lluvia, mantenimiento, lluvia, bus, mantenimiento, bus, bus, cita)
modelo.bake()

In [115]:
print([p.name for p in mantenimiento.parents])  
print([p.name for p in bus.parents])  
print([p.name for p in cita.parents]) 

['lluvia']
['lluvia', 'mantenimiento']
['bus']


In [116]:
print([p.name for p in mantenimiento.children]) 
print([p.name for p in bus.children])  
print([p.name for p in cita.children])  

['bus']
['cita']
[]


In [117]:
# Inferencia con evidencia
predicciones = modelo.predict_proba({"bus": "a tiempo"})

for nodo, prediccion in zip(modelo.nodes, predicciones):
    if isinstance(prediccion, str):
        print(f"{nodo.name}: {prediccion}")
    else:
        print(f"{nodo.name}:")
        for valor, probabilidad in prediccion.probabilities.items():
            print(f"   {valor}: {probabilidad:.2f}")

# Probabilidad de observación completa
print("\nProbabilidad de ['ninguna', 'no', 'a tiempo', 'atendida']:")
print(modelo.probability(["ninguna", "no", "a tiempo", "atendida"]))


lluvia:
   ninguna: 0.79
   suave: 0.17
   fuerte: 0.04
mantenimiento:
   si: 0.44
   no: 0.56
bus: a tiempo
cita:
   atendida: 0.95
   perdida: 0.05

Probabilidad de ['ninguna', 'no', 'a tiempo', 'atendida']:
0.3591


In [119]:
# Nodos sin padres
robo = Node(DiscreteDistribution({
    'v': 0.001,
    'f': 0.999
}), name='Robo')

temblor = Node(DiscreteDistribution({
    'v': 0.001,
    'f': 0.999
}), name='Temblor')

# Nodo Alarma condicionado por Robo y Temblor
alarma = Node(ConditionalProbabilityTable([
    ['v', 'v', 'v', 0.95],
    ['v', 'v', 'f', 0.05],
    ['v', 'f', 'v', 0.94],
    ['v', 'f', 'f', 0.06],
    ['f', 'v', 'v', 0.29],
    ['f', 'v', 'f', 0.71],
    ['f', 'f', 'v', 0.001],
    ['f', 'f', 'f', 0.999],
], [robo, temblor]), name='Alarma')

# Nodo Jorge condicionado por Alarma
jorge = Node(ConditionalProbabilityTable([
    ['v', 'v', 0.90],
    ['v', 'f', 0.10],
    ['f', 'v', 0.05],
    ['f', 'f', 0.95],
], [alarma]), name='Jorge')

# Nodo María condicionado por Alarma
maria = Node(ConditionalProbabilityTable([
    ['v', 'v', 0.70],
    ['v', 'f', 0.30],
    ['f', 'v', 0.01],
    ['f', 'f', 0.99],
], [alarma]), name='María')

# Crear red
red = BayesianNetwork()
red.add_state(robo, temblor, alarma, jorge, maria)

# Conectar bordes según el gráfico
red.add_edges(robo, alarma,
              temblor, alarma,
              alarma, jorge,
              alarma, maria)

red.bake()


In [ ]:
print(red.probability(['f', 'f', 'v', 'v', 'v']))  # P(robo=f, temblor=f, alarma=v, jorge=v, maria=v)

0.00062874063


In [121]:
res = red.predict_proba({'Jorge': 'v'})
for nodo, pred in zip(red.nodes, res):
    print(nodo.name, "→", pred if isinstance(pred, str) else pred.probabilities)


Robo → {'v': 0.016360574479229457, 'f': 0.9836394255207705}
Temblor → {'v': 0.005724428689906382, 'f': 0.9942755713100936}
Alarma → {'v': 0.03863579199206978, 'f': 0.9613642080079301}
Jorge → v
María → {'v': 0.03665869647452815, 'f': 0.9633413035254718}
